In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
d0 = pd.read_csv('./../Data/census/2021/OA_number_of_residents.csv'
                 ).drop(columns={'geography','date','Residence type: Lives in a communal establishment; measures: Value'}
                        ).rename(columns={'geography code':'OA21CD', 'Residence type: Lives in a household; measures: Value':'N_residents_no_community','Residence type: Total; measures: Value':'Total_N_residents'})
d0

,OA21CD,Total_N_residents,N_residents_no_community
0,E00060274,273,273
1,E00060275,416,416
2,E00060276,259,259
3,E00060277,278,278
4,E00060279,276,276
...,...,...,...
188875,W00006938,219,219
188876,W00006940,422,422
188877,W00006941,274,274
188878,W00006942,327,327


In [3]:
d1 = pd.read_csv('./../Data/census/2021/OA_number_of_dwellings.csv').rename(columns={'Observation':'N_dwellings','Output Areas Code':'OA21CD'}).drop(columns='Output Areas')
d2 = pd.read_csv('./../Data/census/2021/OA_N_households_21.csv').rename(columns={'Observation':'N_households','Output Areas Code':'OA21CD'}).drop(columns='Output Areas')
d2

,OA21CD,N_households
0,E00000001,94
1,E00000003,109
2,E00000005,63
3,E00000007,86
4,E00000010,126
...,...,...
188875,W00010693,57
188876,W00010694,158
188877,W00010695,75
188878,W00010696,96


In [4]:
d3 = pd.read_csv('./../Data/census/2021/OA_number_of_household_occupants.csv'
                 ).rename(columns={'Observation':'N_household_occupants', 'Household size (9 categories) Code':'Household_size','Output Areas Code':'OA21CD'}
                          ).drop(columns={'Household size (9 categories)','Output Areas'}
                                 ).pivot(index= 'OA21CD',columns='Household_size', values='N_household_occupants'
                                         ).reset_index()

d3['H_1-2_p'] = d3[1] + d3[2]
d3['H_3-4_p'] = d3[3] + d3[4]
d3['H_5-7_p'] = d3[5] + d3[6] + d3[7]
d3['N_of_people'] = (d3[1]*1) + (d3[2]*2) + (d3[3]*3) + (d3[4]*4) + (d3[5]*5) + (d3[6]*6) + (d3[7]*7) +(d3[8]*8)
d3= d3.drop(columns={0,1,2,3,4,5,6,7}).rename(columns={8:'H_8-more_p'}).loc[:,['OA21CD','H_1-2_p','H_3-4_p','H_5-7_p','H_8-more_p','N_of_people']]
    
d3

Household_size,OA21CD,H_1-2_p,H_3-4_p,H_5-7_p,H_8-more_p,N_of_people
0,E00000001,78,16,0,0,176
1,E00000003,69,36,4,0,256
2,E00000005,53,10,0,0,112
3,E00000007,82,4,1,0,145
4,E00000010,115,7,3,0,173
...,...,...,...,...,...,...
188875,W00010693,52,6,0,0,78
188876,W00010694,92,51,16,0,404
188877,W00010695,41,27,6,0,189
188878,W00010696,56,38,3,0,237


In [5]:
d4 = pd.read_csv('./../Data/census/2021/OA_Number_of_dependent_children.csv'
                 ).rename(columns={'Observation':'N_households','Output Areas Code':'OA21CD'}
                          ).drop(columns={'Output Areas','Ethnic group of the Household Reference Person (8 categories) Code','Ethnic group of the Household Reference Person (8 categories)',
                                          'Dependent child age (6 categories)'}
                                ).groupby(['OA21CD',	'Dependent child age (6 categories) Code']
                                           ).sum().reset_index(
    
                                           ).pivot(index= 'OA21CD',columns='Dependent child age (6 categories) Code', values='N_households').reset_index()

d4['CHILDREN_1_2'] = d4[1]+d4[2]
d4['CHILDREN_3_4'] = d4[3]+d4[4]
d4['CHILDREN_5_more'] = d4[5]

d4 = d4.loc[:,['OA21CD','CHILDREN_1_2','CHILDREN_3_4','CHILDREN_5_more']]

d4

Dependent child age (6 categories) Code,OA21CD,CHILDREN_1_2,CHILDREN_3_4,CHILDREN_5_more
0,E00000001,1,13,2
1,E00000003,10,26,4
2,E00000005,6,2,2
3,E00000007,1,5,0
4,E00000010,1,6,1
...,...,...,...,...
188875,W00010693,0,1,0
188876,W00010694,30,73,9
188877,W00010695,15,27,3
188878,W00010696,36,18,1


In [6]:
d5 = pd.read_csv('./../Data/census/2021/OA_hours_worked.csv'
                 ).rename(columns={'Observation':'N_poeple_hours','Output Areas Code':'OA21CD'}
                          ).drop(columns={'Output Areas','Hours worked (5 categories) Code'}
                                 ).pivot(index= 'OA21CD',columns='Hours worked (5 categories)', values='N_poeple_hours').reset_index()
d5['Part_time_15-30h'] = d5['Part-time: 15 hours or less worked'] + d5['Part-time: 16 to 30 hours worked']
d5['Full_time_31_moreh'] = d5['Full-time: 31 to 48 hours worked'] + d5['Full-time: 49 or more hours worked']
d5['Hours_worked_other'] = d5['Does not apply']

d5 = d5.loc[:,['OA21CD','Part_time_15-30h','Full_time_31_moreh','Hours_worked_other']]
d5

Hours worked (5 categories),OA21CD,Part_time_15-30h,Full_time_31_moreh,Hours_worked_other
0,E00000001,28,60,88
1,E00000003,37,97,122
2,E00000005,12,54,46
3,E00000007,12,94,39
4,E00000010,26,87,65
...,...,...,...,...
188875,W00010693,97,21,623
188876,W00010694,62,143,203
188877,W00010695,20,67,108
188878,W00010696,31,102,101


In [7]:
d6 = pd.read_csv('./../Data/census/2021/OA_Occupation.csv'
                 ).rename(columns={'Output Areas Code':'OA21CD'}
                    ).drop(columns = {'Output Areas','Occupation (current) (10 categories) Code'}
                        ).pivot(index= 'OA21CD',columns='Occupation (current) (10 categories)', values='Observation'
                                ).reset_index()
d6.rename(columns={'Does not apply':'10.Occupation_unspecified'})

d6

Occupation (current) (10 categories),OA21CD,"1. Managers, directors and senior officials",2. Professional occupations,3. Associate professional and technical occupations,4. Administrative and secretarial occupations,5. Skilled trades occupations,"6. Caring, leisure and other service occupations",7. Sales and customer service occupations,"8. Process, plant and machine operatives",9. Elementary occupations,Does not apply
0,E00000001,24,37,13,6,1,4,1,1,0,88
1,E00000003,36,54,32,6,0,4,2,3,0,122
2,E00000005,16,25,15,6,1,1,2,0,0,46
3,E00000007,29,36,29,1,3,4,1,2,1,39
4,E00000010,23,28,28,8,7,9,0,0,11,65
...,...,...,...,...,...,...,...,...,...,...,...
188875,W00010693,5,15,9,8,7,12,26,4,29,623
188876,W00010694,15,51,28,18,24,22,18,13,14,203
188877,W00010695,9,15,13,10,4,10,7,9,8,108
188878,W00010696,19,41,23,12,11,7,7,8,6,101


In [8]:
d7 = pd.read_csv('./../Data/census/2021/OA_Population_density.csv'
                 ).rename(columns={'Output Areas Code':'OA21CD','Observation':'popul_dens'}
                          ).drop(columns={'Output Areas'})
d7

,OA21CD,popul_dens
0,E00000001,26268.7
1,E00000003,60952.4
2,E00000005,12873.6
3,E00000007,1959.2
4,E00000010,71200.0
...,...,...
188875,W00010693,6305.5
188876,W00010694,1561.7
188877,W00010695,980.4
188878,W00010696,629.9


In [9]:
d8 = pd.read_csv('./../Data/census/2021/OA_car_availability.csv'
                 ).drop(columns={'Household composition (8 categories) Code','Household composition (8 categories)','Output Areas','Car or van availability (3 categories) Code'}
                    ).rename(columns={'Output Areas Code':'OA21CD'}
                        ).groupby(['OA21CD','Car or van availability (3 categories)']
                            ).sum().reset_index(
                            ).pivot(index= 'OA21CD',columns='Car or van availability (3 categories)', values='Observation'
                                ).reset_index(
                                ).drop(columns={'Does not apply'}
                                                     ).rename(columns={'1 or more cars or vans in household':'N_H_car_1_more', 'No cars or vans in household':'N_H_no_car'})
d8

Car or van availability (3 categories),OA21CD,N_H_car_1_more,N_H_no_car
0,E00000001,46,46
1,E00000003,57,51
2,E00000005,28,40
3,E00000007,20,66
4,E00000010,19,105
...,...,...,...
188875,W00010693,25,33
188876,W00010694,144,15
188877,W00010695,64,14
188878,W00010696,92,4


In [10]:
d9 = pd.read_csv('./../Data/census/2021/OA_economic_activity_status.csv'
                 ).drop(columns={'Output Areas','Economic activity status (20 categories) Code'}
                        ).rename(columns={'Output Areas Code':'OA21CD'}
                                ).pivot(index= 'OA21CD',columns='Economic activity status (20 categories)', values='Observation'
                                ).reset_index()
d9['N_econ_inactive'] = d9['Economically Inactive: Looking after home or family'] + d9['Economically inactive: Long-term sick or disabled'] + d9['Economically inactive: Other'] + d9['Economically inactive: Retired'] + d9['Economically inactive: Student']
d9['N_Unemployed'] = d9['Economically active (excluding full-time students): Unemployed: Seeking work or waiting to start a job already obtained: Available to start working within 2 weeks']
d9['N_students'] = d9['Economically active and a full-time student: In employment: Employee: Full-time'] + d9['Economically active and a full-time student: In employment: Employee: Part-time'] + d9['Economically active and a full-time student: In employment: Self-employed with employees: Full-time'] + d9['Economically active and a full-time student: In employment: Self-employed with employees: Part-time'] + d9['Economically active and a full-time student: In employment: Self-employed without employees: Full-time'] + d9['Economically active and a full-time student: In employment: Self-employed without employees: Part-time'] + d9['Economically active (excluding full-time students): Unemployed: Seeking work or waiting to start a job already obtained: Available to start working within 2 weeks']
d9['N_FT_empl'] = d9['Economically active (excluding full-time students): In employment: Employee: Full-time'] + d9['Economically active (excluding full-time students): In employment: Self-employed with employees: Full-time'] + d9['Economically active (excluding full-time students): In employment: Self-employed without employees: Full-time']
d9['N_PT_empl'] = d9['Economically active (excluding full-time students): In employment: Employee: Part-time'] + d9['Economically active (excluding full-time students): In employment: Self-employed with employees: Part-time'] + d9['Economically active (excluding full-time students): In employment: Self-employed without employees: Part-time']
d9 = d9.loc[:,['OA21CD','N_econ_inactive','N_Unemployed','N_students','N_FT_empl','N_PT_empl']]

d9

Economic activity status (20 categories),OA21CD,N_econ_inactive,N_Unemployed,N_students,N_FT_empl,N_PT_empl
0,E00000001,63,10,10,60,28
1,E00000003,83,5,6,98,36
2,E00000005,36,3,4,54,11
3,E00000007,32,1,1,94,15
4,E00000010,41,18,19,84,26
...,...,...,...,...,...,...
188875,W00010693,563,1,93,16,9
188876,W00010694,96,4,12,139,55
188877,W00010695,60,7,10,68,17
188878,W00010696,43,1,3,99,28


In [11]:
d10 = pd.read_csv('./../Data/census/2021/OA_general_health.csv'
                 ).drop(columns={'Output Areas','General health (6 categories) Code'}
                        ).rename(columns={'Output Areas Code':'OA21CD'}
                                ).pivot(index= 'OA21CD',columns='General health (6 categories)', values='Observation'
                                ).reset_index()
d10['HE_Bad_or_Very_bad_health'] = d10['Bad health'] + d10['Very bad health']
d10['HE_Good_or_Very_good_health'] = d10['Good health'] + d10['Very good health']
d10 = d10.loc[:,['OA21CD','HE_Bad_or_Very_bad_health','Fair health','HE_Good_or_Very_good_health']].rename(columns={'Fair health':'HE_Fair health'})
d10

General health (6 categories),OA21CD,HE_Bad_or_Very_bad_health,HE_Fair health,HE_Good_or_Very_good_health
0,E00000001,2,26,147
1,E00000003,3,15,238
2,E00000005,3,8,101
3,E00000007,4,6,134
4,E00000010,6,21,147
...,...,...,...,...
188875,W00010693,16,70,654
188876,W00010694,11,38,355
188877,W00010695,14,26,154
188878,W00010696,8,16,208


In [12]:
d11 = pd.read_csv('./../Data/census/2021/OA_housholds_by_deprivation.csv'
                 ).drop(columns={'Output Areas','Household deprivation (6 categories) Code'}
                        ).rename(columns={'Output Areas Code':'OA21CD'}
                                ).pivot(index= 'OA21CD',columns='Household deprivation (6 categories)', values='Observation'
                                ).reset_index()
d11['Household is deprived in one dimension'] = d11['Household is deprived in one dimension'] * 1
d11['Household is deprived in two dimensions'] = d11['Household is deprived in two dimensions'] * 2
d11['Household is deprived in three dimensions'] = d11['Household is deprived in three dimensions'] * 3
d11['Household is deprived in four dimensions'] = d11['Household is deprived in four dimensions'] * 4
d11['D_Household_depriv_points'] = d11['Household is deprived in one dimension'] + d11['Household is deprived in two dimensions']+d11['Household is deprived in three dimensions'] + d11['Household is deprived in four dimensions']
d11 = d11.loc[:,['OA21CD','D_Household_depriv_points','Household is not deprived in any dimension']].rename(columns={'Household is not deprived in any dimension':'D_House_NOT_depriv'})
d11.head()

Household deprivation (6 categories),OA21CD,D_Household_depriv_points,D_House_NOT_depriv
0,E00000001,28,66
1,E00000003,40,74
2,E00000005,19,48
3,E00000007,20,69
4,E00000010,125,40


In [13]:
d12 = pd.read_csv('./../Data/census/2021/OA_education.csv'
                  ).drop(columns={'geography','date','Highest level of qualification: Total: All usual residents aged 16 years and over'}
                         ).rename(columns={'geography code':'OA21CD'}
                                  )

d12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188880 entries, 0 to 188879
Data columns (total 8 columns):
 #   Column                                                                  Non-Null Count   Dtype 
---  ------                                                                  --------------   ----- 
 0   OA21CD                                                                  188880 non-null  object
 1   Highest level of qualification: No qualifications                       188880 non-null  int64 
 2   Highest level of qualification: Level 1 and entry level qualifications  188880 non-null  int64 
 3   Highest level of qualification: Level 2 qualifications                  188880 non-null  int64 
 4   Highest level of qualification: Apprenticeship                          188880 non-null  int64 
 5   Highest level of qualification: Level 3 qualifications                  188880 non-null  int64 
 6   Highest level of qualification: Level 4 qualifications and above        18888

In [14]:
d13 = pd.read_csv('./../Data/census/2021/OA_age.csv'
                  ).drop(columns={'geography','date'}
                         ).rename(columns={'geography code':'OA21CD'}
                                  )
d13['N_children_under_15'] = d13['Age: Aged 4 years and under'] + d13['Age: Aged 5 to 9 years'] + d13['Age: Aged 10 to 14 years']
d13['N_children_under_19'] = d13['Age: Aged 4 years and under'] + d13['Age: Aged 5 to 9 years'] + d13['Age: Aged 10 to 14 years'] + d13['Age: Aged 15 to 19 years']
d13['N_adults_19_64'] = d13['Age: Aged 20 to 24 years'] + d13['Age: Aged 25 to 29 years'] + d13['Age: Aged 30 to 34 years'] + d13['Age: Aged 35 to 39 years'] + d13['Age: Aged 40 to 44 years'] + d13['Age: Aged 45 to 49 years'] + d13['Age: Aged 50 to 54 years'] + d13['Age: Aged 55 to 59 years'] + d13['Age: Aged 60 to 64 years']
d13['N_young_adults_20_39'] = d13['Age: Aged 20 to 24 years'] + d13['Age: Aged 25 to 29 years'] + d13['Age: Aged 30 to 34 years'] + d13['Age: Aged 35 to 39 years'] + d13['Age: Aged 60 to 64 years']
d13['N_mature_adults_40_64'] =   d13['Age: Aged 40 to 44 years'] + d13['Age: Aged 45 to 49 years'] + d13['Age: Aged 50 to 54 years'] + d13['Age: Aged 55 to 59 years']
d13['N_retirees_65_m'] = d13['Age: Aged 65 to 69 years'] + d13['Age: Aged 70 to 74 years'] + d13['Age: Aged 75 to 79 years'] + d13['Age: Aged 80 to 84 years'] + d13['Age: Aged 85 years and over']
d13 = d13.drop(d13.iloc[:,2:20], axis = 1
               ).rename(columns = {'Age: Total':'Total_N_residents(age)'}
                        )
d13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188880 entries, 0 to 188879
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   OA21CD                  188880 non-null  object
 1   Total_N_residents(age)  188880 non-null  int64 
 2   N_children_under_15     188880 non-null  int64 
 3   N_children_under_19     188880 non-null  int64 
 4   N_adults_19_64          188880 non-null  int64 
 5   N_young_adults_20_39    188880 non-null  int64 
 6   N_mature_adults_40_64   188880 non-null  int64 
 7   N_retirees_65_m         188880 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 11.5+ MB


In [15]:
d14 = pd.read_csv('./../Data/census/2021/OA_arrivalUK.csv'
                  ).drop(columns={'geography','date'}
                         ).rename(columns={'geography code':'OA21CD'}
                                  )
d14['N_immigrants_arrived_after18'] = d14.iloc[:,10:20].sum(axis = 1)
d14 = d14.drop(d14.iloc[:,1:20], axis = 1)
d14.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188880 entries, 0 to 188879
Data columns (total 2 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   OA21CD                        188880 non-null  object
 1   N_immigrants_arrived_after18  188880 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.9+ MB


In [16]:
d15 = pd.read_csv('./../Data/census/2021/OA_tenure.csv'
                  ).drop(columns={'geography','date'}
                         ).rename(columns={'geography code':'OA21CD'}
                                  )
d15['N_H_Owned'] = d15.iloc[:,2:7].sum(axis = 1)
d15['N_H_So_Rented'] = d15.iloc[:,7:10].sum(axis = 1)
d15['N_H_Pr_Rented'] = d15.iloc[:,10:14].sum(axis = 1)
d15 = d15.iloc[:,[0,14,15,16]]
d15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188880 entries, 0 to 188879
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   OA21CD         188880 non-null  object
 1   N_H_Owned      188880 non-null  int64 
 2   N_H_So_Rented  188880 non-null  int64 
 3   N_H_Pr_Rented  188880 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 5.8+ MB


In [17]:
d15.iloc[:,2:7]

,N_H_So_Rented,N_H_Pr_Rented
0,14,30
1,4,35
2,4,42
3,2,114
4,140,62
...,...,...
188875,60,48
188876,0,48
188877,2,38
188878,8,28


In [18]:
data = d0.merge(d1, on = 'OA21CD', how = 'left'
                ).merge(d2, on = 'OA21CD', how = 'left'
                        ).merge(d3, on = 'OA21CD', how = 'left'
                                ).merge(d4, on = 'OA21CD', how = 'left'
                                        ).merge(d5, on = 'OA21CD', how = 'left'
                                                ).merge(d6, on = 'OA21CD', how = 'left'
                                                        ).merge(d7, on = 'OA21CD', how = 'left'
                                                                ).merge(d8, on = 'OA21CD', how = 'left'
                                                                        ).merge(d9, on = 'OA21CD', how = 'left'
                                                                                 ).merge(d10, on = 'OA21CD', how = 'left'
                                                                                        ).merge(d11, on = 'OA21CD', how = 'left'
                                                                                                ).merge(d12, on = 'OA21CD', how = 'left'
                                                                                                        ).merge(d13, on = 'OA21CD', how = 'left'
                                                                                                                ).merge(d14, on = 'OA21CD', how = 'left'
                                                                                                                        ).merge(d15, on = 'OA21CD', how = 'left')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188880 entries, 0 to 188879
Data columns (total 57 columns):
 #   Column                                                                  Non-Null Count   Dtype  
---  ------                                                                  --------------   -----  
 0   OA21CD                                                                  188880 non-null  object 
 1   Total_N_residents                                                       188880 non-null  int64  
 2   N_residents_no_community                                                188880 non-null  int64  
 3   N_dwellings                                                             188880 non-null  int64  
 4   N_households                                                            188880 non-null  int64  
 5   H_1-2_p                                                                 188880 non-null  int64  
 6   H_3-4_p                                                             

# Refine

In [19]:
(data['Total_N_residents']-data['Age: Total']).describe()

KeyError: 'Age: Total'

In [19]:
(data['Total_N_residents']-data['N_of_people']).describe()

count    188880.000000
mean          6.009043
std          53.792446
min         -38.000000
25%          -1.000000
50%           0.000000
75%           5.000000
max        4259.000000
dtype: float64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188880 entries, 0 to 188879
Data columns (total 57 columns):
 #   Column                                                                  Non-Null Count   Dtype  
---  ------                                                                  --------------   -----  
 0   OA21CD                                                                  188880 non-null  object 
 1   Total_N_residents                                                       188880 non-null  int64  
 2   N_residents_no_community                                                188880 non-null  int64  
 3   N_dwellings                                                             188880 non-null  int64  
 4   N_households                                                            188880 non-null  int64  
 5   H_1-2_p                                                                 188880 non-null  int64  
 6   H_3-4_p                                                             

In [20]:
listhouse = [5,6,7,8,9,10,11,12,27,28,54,55,56]
listresid = [4,13,14,15,16,17,18,19,20,21,22,23,24,25,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,47,48,49,50,51,52,53]

In [21]:
data_copy = data.copy()

for i in listhouse:
    vector = data_copy.iloc[:,i]
    data_copy[str(data_copy.iloc[:,i].name +'_rate_H')] = vector / data_copy['N_households']

for i in listresid:
    vector = data_copy.iloc[:,i]
    data_copy[str(data_copy.iloc[:,i].name +'_rate_P')] = vector / data_copy['Total_N_residents(age)']

data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188880 entries, 0 to 188879
Columns: 106 entries, OA21CD to N_immigrants_arrived_after18_rate_P
dtypes: float64(50), int64(55), object(1)
memory usage: 152.8+ MB


In [22]:
data_copy.head()

,OA21CD,Total_N_residents,N_residents_no_community,N_dwellings,N_households,H_1-2_p,H_3-4_p,H_5-7_p,H_8-more_p,N_of_people,...,Highest level of qualification: Level 3 qualifications_rate_P,Highest level of qualification: Level 4 qualifications and above_rate_P,Highest level of qualification: Other qualifications_rate_P,N_children_under_15_rate_P,N_children_under_19_rate_P,N_adults_19_64_rate_P,N_young_adults_20_39_rate_P,N_mature_adults_40_64_rate_P,N_retirees_65_m_rate_P,N_immigrants_arrived_after18_rate_P
0,E00060274,273,273,121,112,65,40,7,0,273,...,0.113139,0.102190,0.029197,0.288321,0.343066,0.551095,0.317518,0.233577,0.105839,0.000000
1,E00060275,416,416,166,158,87,55,14,1,414,...,0.169048,0.133333,0.011905,0.252381,0.311905,0.576190,0.357143,0.219048,0.111905,0.011905
2,E00060276,259,259,122,117,81,30,6,0,264,...,0.159533,0.147860,0.019455,0.159533,0.221790,0.505837,0.276265,0.229572,0.272374,0.011673
3,E00060277,278,278,139,122,78,37,6,0,276,...,0.155235,0.122744,0.010830,0.245487,0.299639,0.570397,0.332130,0.238267,0.129964,0.021661
4,E00060279,276,276,136,121,77,37,6,0,272,...,0.185714,0.128571,0.021429,0.185714,0.225000,0.607143,0.325000,0.282143,0.167857,0.010714


# Dictionary:

* N_dwellings - number of individual dwellings on each OA
* N_households - number of individual houseolds in each OA
* popul_dens - population density (number of usual residents per square kilometre)
* Total_N_residents	- number of residents inclusing communal establishments
* N_residents_w/_community - number of residents that dont live in communal establishements

**Household size**
* H_8-more_p = households of more than 8 people
* H_1-2_p = household of 1 or 2 people	
* H_3-4_p = household of 3 or 4 people	
* H_5-7_p = household of 5 to 7 people
* N_of_people = estimate number of people in the area

**Dependent children**
* CHILDREN_1_2	= number of households with 1 or 2 children
* CHILDREN_3_4	= number of households with 3 or 4 children
* CHILDREN_5_more = number of households with 5 and more children

**Hours worked**
* Part_time_15-30h = Number of people in area that are over 16 and work part time between 15-30 hours a week
* Full_time_31_moreh = Number of people in area that are over 16 and work full time 31 or more hours
* Hours_worked_other = number of people over 16, not working or falling outside the defined boundaries

**Occupation**
* 1. Managers, directors and senior officials	
* 2. Professional occupations	
* 3. Associate professional and technical occupations	
* 4. Administrative and secretarial occupations	
* 5. Skilled trades occupations	
* 6. Caring, leisure and other service occupations	
* 7. Sales and customer service occupations	
* 8. Process, plant and machine operatives	
* 9. Elementary occupations

**Car availability**
* N_H_car_1_more = Number of Households with one or more cars	
* N_H_no_car = number of households with no car
- this had a 'do not apply' column however, this was 0 across all OAs so was deleted

**Employment**
* N_econ_inactive = number of people in economic inactive state (retired, willingly not working, on disability or other)
* N_Unemployed = number of unemployed people actively searching for job
* N_students = Number of full-time students in the area, any of them could be unemployed, have part time job or full time job
* N_FT_empl	= Number of full-time employees (both employed and self-employed)
* N_PT_empl = Number of part_time employees (both employed and self-employed)

**Health**
* HE_Bad_or_Very_bad_health - number of people who reported bad or very bad health
* HE_Fair health - number of people who reported fair health
* HE_Good_or_Very_good_health - number of people who reported good or very good health
- There was a column for those who reported 'not applicable' but this was 0 across all OAs so was deleted

**Deprivation**
* D_Household_depriv_points - number of household weighted by the deprivation levels - for example if there is 5 households deprived on 3 levels is 5 * 3 = 15, 
* D_House_NOT_Depriv' - number of households that are not deprived in any dimension
- Ther was a 'do not apply' collumn which had 0s across all areas so it was deleted

**Immigration**
* N_immigrants_arrived_after18 - number of residents in the area that arrived into UK at 18 or after (this is only adults exluding children)

**Age**
 * Total_N_residents(age) - total number of residents taken from teh age dissemination
 * N_children_under_15  - self explanatory
 * N_children_under_19 
 * N_adults_19_64  
 * N_young_adults_20_39    
 * N_mature_adults_40_64  
 * N_retirees_65_m

 **Education**
* Highest level of qualification: No qualifications - number of people that reported X
* Highest level of qualification: Level 1 and entry level qualifications 
* Highest level of qualification: Level 2 qualifications 
* Highest level of qualification: Apprenticeship 
* Highest level of qualification: Level 3 qualifications 
* Highest level of qualification: Level 4 qualifications and above 
* Highest level of qualification: Other qualifications  

**Tenure**
* N_H_Owned - number of househodls in setting where the property is fully or partially owned
* N_H_So_Rented - number of househodls in setting where the property is socially rented
* N_H_Pi_Rented - number of househodls in setting where the property is privately rented

In [ ]:
#data.to_csv('./../Data/census/2021/OA_combined_data.csv')

In [ ]:
data_copy.to_csv('./../Data/census/2021/OA_combined_data_refined.csv')

In [1]:

from statsmodels import api as sm
import numpy


def setup(alpha=0.5, size=(1000, 1)):
    X1 = numpy.random.normal(0, 0.1, size=size)
    # Now, let's create an X2 that is endogenous to X1.
    # Alpha represents the "amount" of X2 that comes from X1.
    # if alpha=0, then X2 is independent of X1. If alpha=1, then
    # the regressoin will fail because X1 and X2 are perfectly
    # collinear. The default mixes them half and half.
    X2 = X1 * alpha + numpy.random.normal(0, 1, size=size) * (1 - alpha)
    y = 1 + X1 * 4 + X2 * 2 + numpy.random.normal(0, 0.1, size=size)
    return y, numpy.column_stack((numpy.ones_like(y), X1, X2))

alpha = .5

y, X = setup(alpha=alpha)

# see that all of the indirect effect of "X1" is assigned to X2?
model = sm.OLS(endog=y, exog=X).fit()
model.summary()

# You can see how this changes the "effect of X1 on X2":
X11 = 0
X12 = 1
X21 = X11 * alpha
X22 = X12 * alpha

# Now, this is what happens for a "single unit change" in X1, when
# X2 is related to X1. We can't *just* look at the X1 difference,
# in X1, since a change in X1 also changes X2. We need both to see
# the "total" effect of X1 on y, not just X1's effect conditional on X2.
# This is kind of endogeneity between X variables is a
# violation of the "ceteris paribus"/all-things-equal
# assumption. Thus, X2 might be a kind of "bad control"
# if we hope to get an accurate picture of the "full" effect of X1 on y
dx1 = X12 - X11
dx2 = X22 - X21

# now, this is the total impact of X1, including the "indirect"
# impact that happens *through* X2
total_effect_of_x1 = model.params[1] * dx1 + model.params[2] * dx2
bias = (total_effect_of_x1 - model.params[1]) / model.params[1]

print(
    f"The total effect of a unit change in X1 on y is {total_effect_of_x1:.4f} "
    f", {bias*100:.2f}% larger than the estimated effect is {model.params[1]:.4f}, arising from"
    f" a {dx1:.2f} change in X1 (which causes an extra {dx2:.2f} change in X2, too)"
)

The total effect of a unit change in X1 on y is 4.9990 , 25.01% larger than the estimated effect is 3.9988, arising from a 1.00 change in X1 (which causes an extra 0.50 change in X2, too)
